[19. Black-Scholes Formula, Risk-neutral Valuation](https://www.youtube.com/watch?v=TnS8kI_KuJc)
[Lecture 11 - Risk-Neutral Valuation](https://www.youtube.com/watch?v=Z5MXNdsLrxM)

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def Call(S,K,T,r,v,d=0):
    
    d1 = (np.log(S/K) + (r-d+0.5*v**2)*T) / (v*np.sqrt(T));
    d2 = d1 - v*np.sqrt(T);

    # https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.norm.html
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

In [3]:
def Put(S,K,T,r,v,d=0):
    
    d1 = (np.log(S/K) + (r-d+0.5*v**2)*T) / (v*np.sqrt(T));
    d2 = d1 - v*np.sqrt(T);

    # https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.norm.html
    return - S*norm.cdf(-d1) + K*np.exp(-r*T)*norm.cdf(-d2)

In [4]:
S=258; K=250; T=1; r=0.03; v=0.20; B=190; # Barrier

In [5]:
Call_1 = Call(S,K,T,r,v); 
Put_1 = Put(S,K,T,r,v); 

print('Call option price by Black Scholes formula : ', Call_1)
print('Put option price by Black Scholes formula  : ', Put_1)

Call option price by Black Scholes formula :  28.564342589288287
Put option price by Black Scholes formula  :  13.175725976415308


In [6]:
M = 12*21; 
t = np.linspace(0, T, num=M+1, endpoint=True);
dt = t[2]-t[1];

In [7]:
# Choose a parametrization
if 0:
    # Wilmott parametrization
    U=1+v*np.sqrt(dt); 
    D=1-v*np.sqrt(dt); 
    q_u=0.5+r*np.sqrt(dt)/(2*v); 
    q_d=1-q_u; 
elif 0:
    # CRR parametrization
    U=exp(v*np.sqrt(dt)); 
    D=exp(-v*np.sqrt(dt)); 
    q_u=(np.exp(r*dt)-D)/(U-D); 
    q_d=1-q_u; 
elif 1:
    # JR parametrization
    U=np.exp((r-0.5*v**2)*dt+v*np.sqrt(dt)); 
    D=np.exp((r-0.5*v**2)*dt-v*np.sqrt(dt)); 
    q_u=0.5; 
    q_d=1-q_u; 

In [8]:
# put payoff at maturity
Stock = S * D**np.arange(0,M+1) * U**np.arange(M,-1,-1);
V = np.maximum(K-Stock,0);

# risk-neutral valuation
for i in range(M-1,-1,-1):
    V = np.exp(-r*dt) * (q_u*V[:-1]+q_d*V[1:]) 

Put_2 = V[0];
print('Put option price by binomial model : ',Put_2 )

Put option price by binomial model :  13.191197925291293


In [9]:
# knock-out barrier put payoff at maturity
Stock = S * D**np.arange(0,M+1) * U**np.arange(M,-1,-1);
V = np.maximum(K-Stock,0);

# risk-neutral valuation
for i in range(M-1,-1,-1):
    Stock = S * D**np.arange(0,i+1) * U**np.arange(i,-1,-1);
    V = np.exp(-r*dt) * (q_u*V[:-1]+q_d*V[1:]) 
    V[Stock<=B] = 0

Put_B = V[0];
print('Barrier put option price : ', Put_B)

Barrier put option price :  6.922756601110634


In [10]:
# American put payoff at maturity
Stock = S * D**np.arange(0,M+1) * U**np.arange(M,-1,-1);
V = np.maximum(K-Stock,0);

# risk-neutral valuation
for i in range(M-1,-1,-1):
    Stock = S * D**np.arange(0,i+1) * U**np.arange(i,-1,-1);
    V = np.exp(-r*dt) * (q_u*V[:-1]+q_d*V[1:]) 
    V = np.maximum(V, K-Stock);

Put_A = V[0];
print('American put option price : ', Put_A)

American put option price :  13.728816831000978
